<a href="https://colab.research.google.com/github/Elwing-Chou/ml0602/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0602/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0602/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f34a4e39710>)

In [75]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")

In [76]:
# 分類: 輸出 0 1 2 3
# series.replace(字典)
# {"李白":0, "杜甫":1, "白居易":2}
# series.unique/series.value_counts
u = train["作者"].unique()
name2index = {n:i for i, n in enumerate(u)}
index2name = {i:n for i, n in enumerate(u)}
print(name2index)
print(index2name)
y_train = train["作者"].replace(name2index)
y_test = test["作者"].replace(name2index)
y_test

{'李白': 0, '杜甫': 1, '白居易': 2}
{0: '李白', 1: '杜甫', 2: '白居易'}


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
Name: 作者, dtype: int64

In [77]:
# series.apply(func不加小括號)
# !!我做過實驗了, 詩詞分類載入大辭典反而適得其反, 但是普通時候你要載
import jieba
def poemcut(p):
    return " ".join(jieba.cut(p))
x_train = train["內容"].apply(poemcut)
x_test = test["內容"].apply(poemcut)
x_test

0      日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1     朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2      李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3       故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4                 危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5                  床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6     天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
7               眾鳥 高 飛 儘 ， 孤雲獨 去 閒 。 相看 兩不厭 ， 隻 有 敬亭山 。
8     鳳凰 台上 鳳凰遊 ， 鳳去 台空江 自流 。 吳宮 花草 埋 幽徑 ， 晉代 衣冠 成古丘...
9     渡遠 荊門外 ， 來 從 楚國遊 。 山 隨 平野 儘 ， 江入 大荒 流 。 月 下 飛天...
10    百川 日東流 ， 客去 亦 不息 。 我 生苦 漂 蕩 ， 何時 有 終極 。 讚 公 釋 ...
11    細泉 兼 輕冰 ， 沮洳 棧道 濕 。 不辭 辛苦 行 ， 迫此 短景急 。 石門 雪雲隘 ...
12    首路 栗亭 西 ， 尚想 鳳凰村 。 季冬 攜 童稚 ， 辛苦 赴 蜀門 。 南登 木皮 嶺...
13    落日 在 簾 鉤 ， 溪邊 春事幽 。 芳菲 緣岸 圃 ， 樵 爨 倚灘 舟 。 啅 雀 爭...
14    竇侍 禦 ， 驥 之子 ， 鳳之雛 。 年 未 三十 忠義俱 ， 骨鯁 絕代 無 。 炯 如...
15    百草 競春華 ， 麗春應 最勝 。 少須 好 顏色 ， 多漫枝條 剩 。 紛紛 桃李 枝 ，...
16    何年 顧虎頭 ， 滿壁畫 瀛州 。 赤 日 石林 氣 ， 青天 江海 流 。 錫飛常 近鶴 ...
17    野寺 隱喬木 ， 山僧 高下 居 。 石門 日色異 ， 絳 氣橫 扶疏 。 窈窕 入

In [78]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(x_train)
len(vec.vocabulary_)

52294

In [79]:
# fit_transform: 測試資料只做transform
# 稀疏矩陣
x_train_count = vec.transform(x_train)
x_test_count = vec.transform(x_test)
x_test_count

<30x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 352 stored elements in Compressed Sparse Row format>

In [80]:
# 檢查標點符號, \r \n
# vec.vocabulary_["。"]
# vec.vocabulary_["\r\n"]

In [81]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [82]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8333333333333334

In [83]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
             columns=["{}(預測)".format(n) for n in u],
             index=["{}(正確)".format(n) for n in u])

,李白(預測),杜甫(預測),白居易(預測)
李白(正確),8,1,1
杜甫(正確),0,9,1
白居易(正確),1,1,8


In [84]:
poem = input("輸入詩:")
poems = [poemcut(poem)]
poems_count = vec.transform(poems)
pre = clf.predict(poems_count)[0]
print("答案是:", index2name[pre])
pre_proba = clf.predict_proba(poems_count)[0]
for name, proba in zip(u, pre_proba):
    print(name, "的機率:", proba)

輸入詩:床前明月光，   疑是地上霜。   舉頭望明月，   低頭思故鄉。
答案是: 白居易
李白 的機率: 1.0858665227667848e-06
杜甫 的機率: 2.3194161760280008e-05
白居易 的機率: 0.9999757199717183
